# Link PICO Research Question to Systematic Review

Creates nanopublications that link PICO research questions to a systematic review.

**Template:** [Linking a research question to a systematic review](https://w3id.org/np/RAkZ5aKVkobNvvtOvattwuTXlvI0eg38ngzs01m9447Eg)

This template uses the Science Live ontology predicate `targetsResearchQuestion` to establish
the relationship between a systematic review and its research question(s).

---

## Instructions

1. **Create a JSON file** with your link details (see template below)
2. **Set the path** to your JSON file in Section 1
3. **Run All Cells** → Get your `.trig` file

---
# 📁 SECTION 1: INPUT FILE (EDIT THIS)
---

In [1]:
# Path to your Link configuration JSON file
LINK_FILE = "../inputs/quantum-biodiversity/link-pico-to-review.json"

---
# ⚙️ SECTION 2: SETUP
---

In [2]:
# Install dependencies (uncomment if needed)
# !pip install nanopub rdflib

In [3]:
import json
from rdflib import Graph, Dataset, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, XSD, FOAF
from datetime import datetime, timezone
from pathlib import Path

# Namespaces (matching Nanodash)
NP = Namespace("http://www.nanopub.org/nschema#")
DCT = Namespace("http://purl.org/dc/terms/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
NPX = Namespace("http://purl.org/nanopub/x/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ORCID = Namespace("https://orcid.org/")

# Science Live ontology
SLV = Namespace("https://w3id.org/sciencelive/o/terms/")

# Template URIs
LINK_TEMPLATE = URIRef("https://w3id.org/np/RAkZ5aKVkobNvvtOvattwuTXlvI0eg38ngzs01m9447Eg")
PROV_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")
PUBINFO_TEMPLATE_3 = URIRef("https://w3id.org/np/RAoTD7udB2KtUuOuAe74tJi1t3VzK0DyWS7rYVAq1GRvw")

print("✓ Setup complete")

✓ Setup complete


---
# 📖 SECTION 3: LOAD & VALIDATE
---

In [4]:
# Load Link configuration from JSON
print(f"Loading: {LINK_FILE}")

with open(LINK_FILE, 'r', encoding='utf-8') as f:
    config = json.load(f)

# Extract fields
AUTHOR_ORCID = config['author']['orcid']
AUTHOR_NAME = config['author']['name']

SYSTEMATIC_REVIEW_URI = config['link']['systematic_review_uri']
RESEARCH_QUESTIONS = config['link']['research_questions']  # List of URIs

OUTPUT_FILENAME = config['output']['filename']

print(f"✓ Loaded configuration")
print(f"  Systematic Review: {SYSTEMATIC_REVIEW_URI}")
print(f"  Research Questions: {len(RESEARCH_QUESTIONS)}")

Loading: ../inputs/quantum-biodiversity/link-pico-to-review.json
✓ Loaded configuration
  Systematic Review: https://w3id.org/spaces/sciencelive/quantum-biodiversity-review
  Research Questions: 1


In [5]:
# Validate
print("Validating...")

errors = []
if not AUTHOR_ORCID:
    errors.append("author.orcid is required")
if not AUTHOR_NAME:
    errors.append("author.name is required")
if not SYSTEMATIC_REVIEW_URI:
    errors.append("link.systematic_review_uri is required")
if not RESEARCH_QUESTIONS or len(RESEARCH_QUESTIONS) == 0:
    errors.append("link.research_questions must have at least one URI")

# Validate URIs
if SYSTEMATIC_REVIEW_URI and not SYSTEMATIC_REVIEW_URI.startswith("http"):
    errors.append("link.systematic_review_uri must be a valid URI starting with http")
for i, rq in enumerate(RESEARCH_QUESTIONS):
    if not rq.startswith("http"):
        errors.append(f"link.research_questions[{i}] must be a valid URI starting with http")

if errors:
    print("❌ Validation errors:")
    for e in errors:
        print(f"   - {e}")
    raise ValueError("Please fix the errors in your JSON file")
else:
    print("✓ All fields valid")

Validating...
✓ All fields valid


---
# 🔨 SECTION 4: BUILD NANOPUBLICATION
---

In [6]:
# Create dataset with named graphs
TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")

this_np = URIRef("http://purl.org/nanopub/temp/np/")
head_graph = URIRef("http://purl.org/nanopub/temp/np/Head")
assertion_graph = URIRef("http://purl.org/nanopub/temp/np/assertion")
provenance_graph = URIRef("http://purl.org/nanopub/temp/np/provenance")
pubinfo_graph = URIRef("http://purl.org/nanopub/temp/np/pubinfo")

# Subject URIs
systematic_review_uri = URIRef(SYSTEMATIC_REVIEW_URI)
author_uri = ORCID[AUTHOR_ORCID]

ds = Dataset()

# Bind prefixes
ds.bind("this", "http://purl.org/nanopub/temp/np/")
ds.bind("sub", TEMP_NP)
ds.bind("np", NP)
ds.bind("dct", DCT)
ds.bind("nt", NT)
ds.bind("npx", NPX)
ds.bind("xsd", XSD)
ds.bind("rdfs", RDFS)
ds.bind("orcid", ORCID)
ds.bind("prov", PROV)
ds.bind("foaf", FOAF)
ds.bind("slv", SLV)

print("✓ Dataset created")

✓ Dataset created


In [7]:
# HEAD
head = ds.graph(head_graph)
head.add((this_np, RDF.type, NP.Nanopublication))
head.add((this_np, NP.hasAssertion, assertion_graph))
head.add((this_np, NP.hasProvenance, provenance_graph))
head.add((this_np, NP.hasPublicationInfo, pubinfo_graph))
print(f"✓ Head: {len(head)} triples")

✓ Head: 4 triples


In [8]:
# ASSERTION - Link systematic review to research question(s)
assertion = ds.graph(assertion_graph)

# Add link for each research question (repeatable statement)
for rq_uri in RESEARCH_QUESTIONS:
    assertion.add((
        systematic_review_uri,
        SLV.targetsResearchQuestion,
        URIRef(rq_uri)
    ))
    print(f"  → Linked: {rq_uri[:60]}..." if len(rq_uri) > 60 else f"  → Linked: {rq_uri}")

print(f"✓ Assertion: {len(assertion)} triples")

  → Linked: https://w3id.org/np/RASr_5SP0NhXfz43auGCrxon4_kUUj0AfA56gW13...
✓ Assertion: 1 triples


In [9]:
# PROVENANCE
provenance = ds.graph(provenance_graph)
provenance.add((assertion_graph, PROV.wasAttributedTo, author_uri))
print(f"✓ Provenance: {len(provenance)} triples")

✓ Provenance: 1 triples


In [10]:
# PUBINFO
pubinfo = ds.graph(pubinfo_graph)
now = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")

pubinfo.add((author_uri, FOAF.name, Literal(AUTHOR_NAME)))
pubinfo.add((this_np, DCT.created, Literal(now, datatype=XSD.dateTime)))
pubinfo.add((this_np, DCT.creator, author_uri))
pubinfo.add((this_np, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))
pubinfo.add((this_np, NPX.wasCreatedAt, URIRef("https://nanodash.knowledgepixels.com/")))

# Generate label based on template pattern: "${systematicReview} targets ${researchQuestion}"
# Extract short names for label
review_short = SYSTEMATIC_REVIEW_URI.split('/')[-1]
rq_count = len(RESEARCH_QUESTIONS)
if rq_count == 1:
    rq_short = RESEARCH_QUESTIONS[0].split('/')[-1]
    label = f"{review_short} targets {rq_short}"
else:
    label = f"{review_short} targets {rq_count} research questions"

# Truncate if needed
if len(label) > 100:
    label = label[:97] + "..."
pubinfo.add((this_np, RDFS.label, Literal(label)))

# Template references
pubinfo.add((this_np, NT.wasCreatedFromProvenanceTemplate, PROV_TEMPLATE))
pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))
pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_3))
pubinfo.add((this_np, NT.wasCreatedFromTemplate, LINK_TEMPLATE))

print(f"✓ Pubinfo: {len(pubinfo)} triples")

✓ Pubinfo: 11 triples


---
# 📄 SECTION 5: OUTPUT
---

In [11]:
# Serialize and save
trig_output = ds.serialize(format="trig")

output_path = Path(f"{OUTPUT_FILENAME}.trig")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(trig_output)

print(f"✓ Saved to: {output_path.absolute()}")

✓ Saved to: /Users/annef/Documents/FAIR2Adapt/systematic-review-pipeline/notebooks/link-pico-quantum-biodiversity.trig


In [12]:
# Display output
print("=" * 70)
print("NANOPUBLICATION (TriG format)")
print("=" * 70)
print(trig_output)

NANOPUBLICATION (TriG format)
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix slv: <https://w3id.org/sciencelive/o/terms/> .
@prefix sub: <http://purl.org/nanopub/temp/np/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0000-0002-1784-2920 .
}

sub:assertion {
    <https://w3id.org/spaces/sciencelive/quantum-biodiversity-review> slv:targetsResearchQuestion <https://w3id.org/np/RASr_5SP0NhXfz43auGCrxon4_kUUj0AfA56gW13Yfqak> .
}

sub:Head {
    sub: a np:Nanopublication ;
        np:hasAssertion sub:assertion ;
        np:hasProvenance sub:provenance ;
        np:hasPublicationInfo sub:pubinfo .
}


In [13]:
# Summary
print("=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Input:              {LINK_FILE}")
print(f"Output:             {output_path}")
print(f"Author:             {AUTHOR_NAME} (orcid:{AUTHOR_ORCID})")
print()
print("Link Details:")
print(f"  Systematic Review: {SYSTEMATIC_REVIEW_URI}")
print(f"  Research Questions: {len(RESEARCH_QUESTIONS)}")
for i, rq in enumerate(RESEARCH_QUESTIONS, 1):
    print(f"    {i}. {rq}")
print()
print(f"Template: https://w3id.org/np/RAkZ5aKVkobNvvtOvattwuTXlvI0eg38ngzs01m9447Eg")
print()
print("Next steps:")
print(f"  Sign:    nanopub sign {output_path}")
print(f"  Publish: nanopub publish {output_path.stem}.signed.trig")

SUMMARY
Input:              ../inputs/quantum-biodiversity/link-pico-to-review.json
Output:             link-pico-quantum-biodiversity.trig
Author:             Anne Fouilloux (orcid:0000-0002-1784-2920)

Link Details:
  Systematic Review: https://w3id.org/spaces/sciencelive/quantum-biodiversity-review
  Research Questions: 1
    1. https://w3id.org/np/RASr_5SP0NhXfz43auGCrxon4_kUUj0AfA56gW13Yfqak

Template: https://w3id.org/np/RAkZ5aKVkobNvvtOvattwuTXlvI0eg38ngzs01m9447Eg

Next steps:
  Sign:    nanopub sign link-pico-quantum-biodiversity.trig
  Publish: nanopub publish link-pico-quantum-biodiversity.signed.trig


---
# 🚀 SECTION 6: SIGN & PUBLISH (OPTIONAL)
---

In [14]:
PUBLISH = True
USE_TEST_SERVER = False

In [15]:
if PUBLISH:
    from nanopub import Nanopub, NanopubConf, load_profile
    
    profile = load_profile()
    print(f"Loaded profile: {profile.name}")
    
    conf = NanopubConf(profile=profile, use_test_server=USE_TEST_SERVER)
    np_obj = Nanopub(rdf=output_path, conf=conf)
    
    np_obj.sign()
    print(f"✓ Signed")
    
    signed_path = Path(f"{OUTPUT_FILENAME}.signed.trig")
    np_obj.store(signed_path)
    print(f"✓ Saved: {signed_path}")
    
    np_obj.publish()
    print(f"✓ Published: {np_obj.source_uri}")
else:
    print("Publishing disabled. Set PUBLISH = True to enable.")

Loaded profile: Anne Fouilloux
✓ Signed
✓ Saved: link-pico-quantum-biodiversity.signed.trig
✓ Published: https://w3id.org/np/RAfL0kcbhPK_dy08m8EQr2ppIPmvdLIuVhy-2vgH6w42c


---
# 📋 JSON TEMPLATE

Create a JSON file with this structure:

```json
{
  "author": {
    "orcid": "0000-0000-0000-0000",
    "name": "Your Name"
  },
  "link": {
    "systematic_review_uri": "https://w3id.org/spaces/sciencelive/your-review",
    "research_questions": [
      "https://w3id.org/np/RA..."
    ]
  },
  "output": {
    "filename": "link-pico-to-review"
  }
}
```

## Fields

| Field | Required | Description |
|-------|----------|-------------|
| `author.orcid` | Yes | Your ORCID identifier |
| `author.name` | Yes | Your full name |
| `link.systematic_review_uri` | Yes | URI of your systematic review |
| `link.research_questions` | Yes | List of PICO research question nanopub URIs |
| `output.filename` | Yes | Output filename (without .trig extension) |

## Multiple Research Questions

You can link multiple research questions to a single systematic review:

```json
{
  "link": {
    "systematic_review_uri": "https://w3id.org/spaces/sciencelive/my-review",
    "research_questions": [
      "https://w3id.org/np/RA_question_1...",
      "https://w3id.org/np/RA_question_2...",
      "https://w3id.org/np/RA_question_3..."
    ]
  }
}
```

## Semantic Relationship

This template creates the following RDF assertion:

```turtle
<systematic_review_uri> slv:targetsResearchQuestion <research_question_uri> .
```

Where `slv:` = `https://w3id.org/sciencelive/o/terms/`

---